# 整理数据集

In [36]:
import shutil
import os
import pandas as pd

In [37]:
data_dir='data'
train_dir='train'
test_dir='test'

In [2]:
def mkdir_if_not_exist(path):
    if not os.path.exists(os.path.join(*path)):
        os.makedirs(os.path.join(*path))

for file in os.listdir(os.path.join(data_dir,train_dir)):
    label=file.split('.')[0][0]
    mkdir_if_not_exist([data_dir,'train2',label])
    shutil.copy(
        os.path.join(data_dir,train_dir,file),
        os.path.join(data_dir,'train2',label)
    )

In [7]:
for file in os.listdir(os.path.join(data_dir,test_dir)):
    label=file.split('.')[0][0]
    mkdir_if_not_exist([data_dir,'test2',label])
    shutil.copy(
        os.path.join(data_dir,train_dir,file),
        os.path.join(data_dir,'test2',label)
    )

# 图像预处理

In [38]:
from mxnet import nd
from mxnet import image
from mxnet import gluon

train_augs = [
    image.HorizontalFlipAug(.5),
    image.RandomCropAug((224,224))
]

test_augs = [
    image.CenterCropAug((224,224))
]

def transform(data, label, augs):
    data = data.astype('float32')
    for aug in augs:
        data = aug(data)
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

In [39]:
train_imgs = gluon.data.vision.ImageFolderDataset(
    data_dir+'/'+'train2',
    transform=lambda X, y: transform(X, y, train_augs))
test_imgs = gluon.data.vision.ImageFolderDataset(
    data_dir+'/'+'test2',
    transform=lambda X, y: transform(X, y, test_augs))

data = gluon.data.DataLoader(train_imgs, 32, shuffle=True)

# 构造网络

In [40]:
# net = gluon.nn.HybridSequential()
# with net.name_scope():
#     net.add(gluon.nn.Dense(120))
#     net.add(gluon.nn.Flatten())
#     net.add(gluon.nn.Dense(10))
# net.initialize()

In [41]:
from mxnet.gluon.model_zoo import vision as models

pretrained_net = models.resnet18_v2(pretrained=True)

In [42]:
from mxnet import init

finetune_net = models.resnet18_v2(classes=10)
finetune_net.features = pretrained_net.features
finetune_net.output.initialize(init.Xavier())


In [43]:
finetune_net

ResNetV2(
  (features): HybridSequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (1): BasicBlockV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_siz

# 评估函数

In [44]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

In [45]:
import mxnet as mx
def evaluate_accuracy(data_iterator, net, ctx=[mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc = nd.array([0])
    n = 0.
    if isinstance(data_iterator, mx.io.MXDataIter):
        data_iterator.reset()
    for batch in data_iterator:
        data, label, batch_size = _get_batch(batch, ctx)
        for X, y in zip(data, label):
            acc += nd.sum(net(X).argmax(axis=1)==y).copyto(mx.cpu())
        acc.wait_to_read() # don't push too many operators into backend
        n += batch_size
    return acc.asscalar() / n


# 训练

In [46]:
%matplotlib inline
import matplotlib.pyplot as plt
from mxnet import autograd

In [47]:
def train(net, ctx, batch_size=8, epochs=10, learning_rate=0.01, wd=0.001):
    train_data = gluon.data.DataLoader(train_imgs, batch_size, shuffle=True)
    test_data = gluon.data.DataLoader(test_imgs, batch_size)

    # 确保net的初始化在ctx上
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    sceloss = gluon.loss.SoftmaxCrossEntropyLoss()
    # 训练
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {
        'learning_rate': learning_rate, 'wd': wd})
#     utils.train(train_data, test_data, net, loss, trainer, ctx, epochs)
    for epoch in range(epochs):
        print('start train epoch %d\n'%(epoch))
        train_acc=[]
        train_loss=[]
        total_loss=0.
        total_acc=0.
        for img,label in train_data:
            img,label=img.as_in_context(ctx),label.as_in_context(ctx)
            with autograd.record():
                output=net(img)
                loss=sceloss(output,label)
            loss.backward()
            trainer.step(batch_size)
            
            total_acc+=accuracy(output,label)
            total_loss+=nd.mean(loss).asscalar()
            train_acc.append(total_acc)
            train_loss.append(total_loss)
        print('epoch : %d ; acc : %f ; loss : %f'%(epoch,acc/len(train_data),total_loss/len(train_data)))
    
    evaluate_accuracy(ctx=ctx,data_iterator=test_data,net=net)
    
    _,(fig1,fig2)=plt.subplots((2,1))
    fig1.plot(train_acc,label='acc')
    fig1.legend()
    fig2.plot(train_loss,label='loss')
    fig2.legend()
    plt.show()
            

In [ ]:
train(net=pretrained_net,ctx=mx.gpu())